In [1]:
from openai import OpenAI
from prompts import colloquial_style_aug_sys_prompt
from tqdm import tqdm

import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [1]:
def colloquial_augment(queries: dict[str, list[str]]):
    """Augment existing queries to adopt a colloquial language style."""
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": colloquial_style_aug_sys_prompt},
            {"role": "user", "content": json.dumps(queries, indent=4)},
        ],
        response_format={"type": "json_object"},
    )
    return res.choices[0].message.content

In [4]:
def persist(data: dict, file):
    """Persist data to the given file."""
    json.dump(data, file, ensure_ascii=False)
    file.write('\n')

In [5]:
def main(query_file_name: str):
    query_file = open(query_file_name, 'r')
    persist_file = open("colloquial.jsonl", 'a')
    for line in tqdm(query_file):
        temp = json.loads(line)
        queries = {"customer_queries": temp["customer_queries"]}
        colloquial = colloquial_augment(queries)
        
        colloquial = json.loads(colloquial)
        colloquial.update({"title": temp["title"], "tone": temp["tone"], "style": "colloquial"})

        persist(colloquial, persist_file)

In [6]:
main("queries.jsonl")
main("product_aug.jsonl")

53it [03:31,  4.00s/it]
13it [02:17, 10.57s/it]


In [9]:
main("tone_augmented.jsonl")

132it [14:48,  6.73s/it]


In [12]:
n = 0
with open("colloquial.jsonl", 'r') as f:
    for json_str in f:
        obj = json.loads(json_str)
        n += len(obj['customer_queries'])

In [13]:
print(n)

3898
